<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1a5d33d670d3ea1d872b51b19909f798eed28d57a9f9bf6e639ebcf830a54bc9
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-11 14:19:50
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: 45.03 K (0.32%)
Current PnL: -28.87 L (-18.65%)
CY Booked + Current PnL: -14.58 L (-9.42%)
-------------------
Total profit:  1.22 L
Total loss:  -30.09 L
-------------------
Total Booked + Current PnL: 12.40 L (9.7%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.25%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.50 L (67.07%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.91,-16.64,20.08,0.09,16868.0,-16773.0,84004.0,94.88,43.0,M-SC,2.37,253.0,-0.99,0.62,10.36,OX40N,NTT,DURABLES
50,MASFIN,398.61,0.20,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,56.0,H-SC,7.10,164.0,-0.08,0.71,39.54,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.79,-13.56,15.70,0.02,23691.0,-23663.0,150896.0,129.91,49.0,M-SC,16.16,234.0,-1.00,1.12,31.25,OX40N,NTT,PAINTS
43,ITC,452.00,-0.07,-1.57,12.20,10.44,24033.0,-3144.0,196994.0,-41.15,46.0,X-LC,1.75,5.0,-0.13,1.47,3.90,X40,NTT,FMCG
82,VOLTAS,1530.00,0.37,4.90,14.10,19.69,28360.0,9393.0,201135.0,-5.36,45.0,X-MC,2.57,78.0,0.33,1.50,12.03,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,3.83,-17.64,47.63,21.58,54973.0,-24723.0,115416.0,-58.46,27.0,X-MC,7.12,56.0,-0.45,0.86,3.83,X40N,ATH,IT
18,COFFEEDAY,80.00,3.41,-43.10,137.74,35.27,88991.0,-48941.0,64608.0,-55.73,36.0,L-SC,10.74,270.0,-0.55,0.48,58.13,XR,NTT,HOTELS
64,SHALBY,327.00,2.98,-24.74,66.23,25.10,90285.0,-44823.0,136320.0,901.59,31.0,M-SC,11.00,250.0,-0.50,1.01,9.25,XY24,NTT,HEALTHCARE
67,SONACOMS,806.63,2.83,-16.11,66.35,39.54,56303.0,-16301.0,84858.0,-34.36,47.0,M-SC,4.58,220.0,-0.29,0.63,19.66,AR,ATH,AUTO
28,GREENPANEL,537.00,2.72,-36.84,126.62,43.14,146418.0,-67442.0,115636.0,159.04,34.0,M-SC,9.48,240.0,-0.46,0.86,8.77,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,-2.16,-4.29,26.57,21.14,57813.0,-9753.0,217586.0,-11.03,38.0,X-MC,4.16,74.0,-0.17,1.62,22.64,XY24,NTT,PAINTS
14,CAMPUS,393.0,-2.16,-10.87,49.54,33.28,74079.0,-18242.0,149533.0,-28.17,39.0,M-SC,2.80,222.0,-0.25,1.11,15.90,XY24,NTT,FOOTWEAR
16,CERA,9475.0,-1.50,-31.03,79.62,23.89,96599.0,-54578.0,121325.0,-33.41,31.0,H-SC,9.74,157.0,-0.56,0.90,4.78,OX40N,NTT,CERAMICS
70,SYMPHONY,1306.0,-1.33,-33.44,50.20,-0.03,57180.0,-57237.0,113904.0,-46.65,44.0,M-SC,16.80,206.0,-1.00,0.85,1.41,OX40N,NTT,DURABLES
11,BATAINDIA,2096.0,-1.27,-38.06,118.30,35.20,94276.0,-48978.0,79692.0,4.55,34.0,X-SC,15.68,92.0,-0.52,0.59,1.23,X40,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.72,-6.87,123.25,107.91,169470.0,-10143.0,137501.0,-24.20,39.0,M-SC,12.36,216.0,-0.06,1.02,2.52,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.79,-13.56,15.70,0.02,23691.0,-23663.0,150896.0,129.91,49.0,M-SC,16.16,234.0,-1.00,1.12,31.25,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.91,-16.64,20.08,0.09,16868.0,-16773.0,84004.0,94.88,43.0,M-SC,2.37,253.0,-0.99,0.62,10.36,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.56,-25.45,52.22,13.47,104975.0,-68643.0,201024.0,-69.28,30.0,H-SC,4.18,158.0,-0.65,1.50,2.21,XY24,NTT,PAINTS
66,SIS,528.00,-0.76,-25.45,62.66,21.26,51866.0,-28259.0,82773.0,1951.30,43.0,H-SC,4.62,166.0,-0.54,0.62,11.83,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.24,6.07,62.41,72.27,112172.0,10286.0,179734.0,-8.8,66.0,M-LC,1.93,99.0,0.09,1.34,12.62,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.24,6.07,62.41,72.27,112172.0,10286.0,179734.0,-8.80,66.0,M-LC,1.93,99.0,0.09,1.34,12.62,XR,NTT,IT
38,INDIAMART,4810.62,0.24,-3.74,114.76,106.72,136243.0,-4616.0,118720.0,-52.34,32.0,H-SC,1.99,139.0,-0.03,0.88,18.76,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.14,-2.47,39.07,35.63,79856.0,-5182.0,204392.0,-16.30,39.0,H-MC,3.15,119.0,-0.06,1.52,13.79,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.72,-6.87,123.25,107.91,169470.0,-10143.0,137501.0,-24.20,39.0,M-SC,12.36,216.0,-0.06,1.02,2.52,OX40N,ATH,CABLES
37,IEX,219.00,2.00,-3.45,53.94,48.63,104130.0,-6907.0,193047.0,-35.77,50.0,H-SC,14.05,136.0,-0.07,1.44,7.70,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.35,-21.57,89.28,48.45,140402.0,-43246.0,157260.0,-27.18,25.0,X-MC,12.19,64.0,-0.31,1.17,2.39,X40N,ATH,FINANCE
22,DIXON,18931.72,3.83,-17.64,47.63,21.58,54973.0,-24723.0,115416.0,-58.46,27.0,X-MC,7.12,56.0,-0.45,0.86,3.83,X40N,ATH,IT
33,HINDUNILVR,2922.00,-0.00,-10.47,26.96,13.67,62051.0,-26903.0,230160.0,-17.11,29.0,X-LC,2.63,14.0,-0.43,1.71,8.66,XY25,NTT,FMCG
23,DMART,5391.45,0.13,-9.20,41.10,28.12,78524.0,-19352.0,191055.0,-21.67,30.0,X-LC,4.95,19.0,-0.25,1.42,12.25,X40N,ATH,FMCG
75,TMPV,600.00,1.18,-17.67,72.69,42.18,165428.0,-48830.0,227580.0,-26.74,30.0,X-LC,6.03,3.0,-0.30,1.69,1.18,XY24,NTT,AUTO


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,2.26,-80.24,531.65,24.79,264533.0,-202099.0,49757.0,-81.09,44.0,X-SC,1.26,86.0,-0.76,0.37,21.64,X40N,NTT,MISC
8,AWL,485.0,0.08,-21.72,96.08,53.50,226959.0,-65532.0,236219.0,-62.10,33.0,X-MC,1.71,58.0,-0.29,1.76,4.92,XY24,NTT,FMCG
43,ITC,452.0,-0.07,-1.57,12.20,10.44,24033.0,-3144.0,196994.0,-41.15,46.0,X-LC,1.75,5.0,-0.13,1.47,3.90,X40,NTT,FMCG
36,ICICIPRULI,790.0,-1.21,5.28,24.40,30.97,45401.0,9332.0,186070.0,-19.54,59.0,X-MC,2.01,75.0,0.21,1.38,18.50,X40,ATH,INSURANCE
3,ACC,3906.0,-0.20,-25.25,119.60,64.15,212733.0,-60081.0,177870.0,-56.31,34.0,X-SC,2.11,82.0,-0.28,1.32,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.20,-25.25,119.60,64.15,212733.0,-60081.0,177870.0,-56.31,34.0,X-SC,2.11,82.0,-0.28,1.32,0.00,XY24,BTT,CEMENT
54,PGHH,17907.65,-0.17,-6.05,42.37,33.76,79939.0,-12150.0,188670.0,-33.72,33.0,X-MC,5.23,57.0,-0.15,1.40,0.00,X40,ATH,FMCG
31,HAVELLS,2069.16,0.41,-10.65,48.26,32.48,141438.0,-34915.0,293076.0,-15.19,37.0,X-MC,5.51,67.0,-0.25,2.18,0.41,X40,ATH,ELECTRICAL
53,PAGEIND,51605.08,1.17,-9.31,39.00,26.07,57915.0,-15240.0,148500.0,-33.27,34.0,X-MC,12.13,55.0,-0.26,1.10,1.17,X40,ATH,APPARELS
75,TMPV,600.00,1.18,-17.67,72.69,42.18,165428.0,-48830.0,227580.0,-26.74,30.0,X-LC,6.03,3.0,-0.30,1.69,1.18,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.04,-29.78,103.43,42.84,47212.0,-19360.0,45646.0,-54.27,41.0,X-SC,36.53,83.0,-0.41,0.34,3.27,XY24,NTT,MISC
15,CAMS,4762.00,2.26,-80.24,531.65,24.79,264533.0,-202099.0,49757.0,-81.09,44.0,X-SC,1.26,86.0,-0.76,0.37,21.64,X40N,NTT,MISC
59,RELAXO,1176.00,-1.24,-47.34,189.23,52.31,144649.0,-68719.0,76441.0,-43.96,43.0,X-SC,6.42,91.0,-0.48,0.57,2.32,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.27,-38.06,118.30,35.20,94276.0,-48978.0,79692.0,4.55,34.0,X-SC,15.68,92.0,-0.52,0.59,1.23,X40,NTT,FOOTWEAR
22,DIXON,18931.72,3.83,-17.64,47.63,21.58,54973.0,-24723.0,115416.0,-58.46,27.0,X-MC,7.12,56.0,-0.45,0.86,3.83,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.19,-13.51,37.91,19.28,113431.0,-46745.0,299211.0,-24.60,58.0,X-LC,5.22,1.0,-0.41,2.23,10.61,X40,ATH,IT
41,INFY,2275.00,0.31,8.22,43.11,54.87,145973.0,25718.0,338606.0,-15.65,61.0,X-LC,3.22,2.0,0.18,2.52,17.18,X40,BTT,IT
75,TMPV,600.00,1.18,-17.67,72.69,42.18,165428.0,-48830.0,227580.0,-26.74,30.0,X-LC,6.03,3.0,-0.30,1.69,1.18,XY24,NTT,AUTO
81,VBL,671.64,0.08,-4.38,41.89,35.67,126507.0,-13845.0,301997.0,-15.77,54.0,X-LC,4.60,4.0,-0.11,2.25,8.86,X40N,ATH,FMCG
43,ITC,452.00,-0.07,-1.57,12.20,10.44,24033.0,-3144.0,196994.0,-41.15,46.0,X-LC,1.75,5.0,-0.13,1.47,3.90,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.19,-17.94,121.40,81.67,93453.0,-16831.0,76979.0,6916.67,39.0,L-SC,13.84,271.0,-0.18,0.57,49.07,XR,NTT,CERAMICS
50,MASFIN,398.61,0.20,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,56.0,H-SC,7.10,164.0,-0.08,0.71,39.54,XR,ATH,FINANCE
13,BSOFT,831.70,1.42,-22.21,93.82,50.79,102658.0,-31233.0,109420.0,-0.27,67.0,H-SC,5.86,151.0,-0.30,0.81,27.40,XR,ATH,IT
39,INDIGOPNTS,1408.00,-0.79,-13.56,15.70,0.02,23691.0,-23663.0,150896.0,129.91,49.0,M-SC,16.16,234.0,-1.00,1.12,31.25,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.62,-2.59,24.16,20.95,59254.0,-6512.0,245256.0,-3.58,47.0,X-LC,18.67,31.0,-0.11,1.82,32.13,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.42,-22.21,93.82,50.79,102658.0,-31233.0,109420.0,-0.27,67.0,H-SC,5.86,151.0,-0.30,0.81,27.40,XR,ATH,IT
56,RAJESHEXPO,518.00,0.87,-62.93,169.96,0.07,86421.0,-86329.0,50848.0,1763.64,57.0,L-SC,2.38,268.0,-1.00,0.38,26.46,OX40N,NTT,JEWELLERY
84,WIPRO,420.00,0.24,6.07,62.41,72.27,112172.0,10286.0,179734.0,-8.80,66.0,M-LC,1.93,99.0,0.09,1.34,12.62,XR,NTT,IT
0,5PAISA,593.00,1.88,-41.20,92.00,12.90,118203.0,-90014.0,128482.0,88.70,44.0,H-SC,11.64,161.0,-0.76,0.96,6.83,OX40N,NTT,FINANCE
32,HCLTECH,1922.01,0.05,8.17,15.31,24.73,40064.0,19764.0,261688.0,12.76,63.0,X-LC,6.20,7.0,0.49,1.95,24.65,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.75
1,25,45.74
2,50,76.67


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.07
MC    30.26
LC    24.66
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.31
X40      22.42
X40N     10.83
XR       10.14
XY25      9.39
AR        9.36
OX40N     7.77
SR        1.02
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.58
X-MC    23.50
X-LC    21.08
M-SC    12.11
X-SC     6.95
H-MC     5.01
L-SC     1.43
M-MC     1.40
M-LC     1.34
H-LC     1.18
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.38,-7.71,42.54
IT,12.76,-17.53,81.07
FINANCE,9.59,-20.18,70.96
ELECTRICAL,6.24,-12.01,53.04
MISC,6.12,-57.98,122.69
PAINTS,6.06,-13.33,30.16
INSURANCE,4.61,-1.90,37.81
PHARMA,4.12,-0.95,33.26
AUTO,3.42,-25.01,78.61


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3230593.0,21
AR,1313454.0,10
XR,1297214.0,13
X40,1023379.0,14
X40N,993781.0,9
OX40N,746207.0,10
XY25,378793.0,6
SR,283199.0,2
MH,83035.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3672849.0,25
M-SC,1427874.0,15
X-MC,1323368.0,16
X-SC,985477.0,8
X-LC,929597.0,11
H-MC,405583.0,3
L-SC,268865.0,3
M-LC,112172.0,1
H-LC,75081.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1285483.0      6
           AR         901110.0      5
M-SC       XY24       812993.0      6
H-SC       XR         773933.0      7
X-SC       X40N       507147.0      3
X-MC       X40        458064.0      7
           XY24       398006.0      3
X-LC       X40        382755.0      5
H-SC       OX40N      346089.0      4
M-SC       OX40N      313697.0      5
X-SC       XY24       295770.0      3
H-SC       SR         283199.0      2
X-MC       X40N       281603.0      4
X-LC       XY24       242566.0      2
H-MC       AR         209808.0      2
X-LC       X40N       205031.0      2
H-MC       XY24       195775.0      1
X-MC       XY25       185695.0      2
X-SC       X40        182560.0      2
L-SC       XR         182444.0      2
M-SC       XR         173729.0      2
           AR         127455.0      2
M-LC       XR         112172.0      1
X-LC       XY25        99245.0      2
L-SC       OX40N       86421.0      1
H-SC       MH          83035.0      1
H-LC       AR          75081.0      1
M-MC       XY25        56769.0      1
L-MC       XR          54936.0      1
L-LC       XY25        37084.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
